In [1]:
sc.install_pypi_package("pandas") #Install pandas version 0.25.1 
sc.install_pypi_package("matplotlib", "https://pypi.org/simple") 
sc.install_pypi_package("IPython", "https://pypi.org/simple") 
sc.install_pypi_package("boto3")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1586650214723_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached pandas-1.0.3-cp36-cp36m-manylinux1_x86_64.whl (10.0 MB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)

  Using cached matplotlib-3.2.1-cp36-cp36m-manylinux1_x86_64.whl (12.4 MB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached kiwisolver-1.2.0-cp36-cp36m-manylinux1_x86_64.whl (88 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)

  Using cached ipython-7.13.0-py3-none-any.whl (780 kB)
  Using cached traitlets-4.3.3-py2.py3-none-any.whl (75 kB)
  Using cached pexpect-4.8.0-py2.py3-none-any.whl (59 kB)
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Using cached Pygments-2.6.1-py3-none-any.whl (914 kB)
  Using cached pickleshare-0.7.5-py2.py3-none-any.whl (6.9 kB)
Processing /mnt/var/lib/livy/.cache/pip/wheels/b4/cb/f1/d142b3bb45d488612cf3943d8a1db090eb95e6687045ba61d1/backcall-0.1.0-py3-none-any.whl
  Using cached prompt_toolkit-3.0.5-py3-none-any.whl (351 kB)
  Using cached jedi-0.16.0-py2.py3-non

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import boto3
from io import StringIO
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col
import pandas as pd
from pyspark.sql.types import FloatType, ArrayType, StringType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
client = boto3.client('s3', region_name='us-east-1')
bucket = 'recommendation-system-md' # already created on S3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
triplets_file_path ='s3a://recommendation-system-md/tripletas.csv'

song_df = spark.read.csv(triplets_file_path, header='true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# User-User

In [5]:
user_mean = song_df.groupBy('user_id').agg(mean('listen_count').alias("media"))
song_mean = song_df.join(user_mean,on='user_id', how='inner')
song_mean = song_mean.withColumn('count_mean',col('listen_count')/col('media'))
only_jams_full=song_mean.filter(song_mean.listen_count>song_mean.media).select(['user_id','song_id'])
(only_jams, only_jams_test) = only_jams_full.randomSplit([0.9, 0.1], 42)
# only_jams = only_jams.filter(only_jams.listen_count>only_jams.media).select(['user_id','song_id'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
def news(s1,s2):
    if s1 is None:
        aux = s2
    else:
        aux = list(set(s2).difference(set(s1)))
    return aux

news_udf = udf(lambda y: news(y[0],y[1]), ArrayType(StringType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
train  = only_jams.groupBy("user_id").agg(collect_set("song_id").alias('Train')).select(['user_id','Train'])
test = only_jams_test.groupBy("user_id").agg(collect_set("song_id").alias('Test')).select(['user_id','Test'])
all_songs = song_mean.groupBy("user_id").agg(collect_set("song_id").alias('All')).select(['user_id','All'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
union_all = all_songs.join(test, how = "left", on = 'user_id').join(train, how = "left", on = 'user_id')
union_all.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+
|             user_id|                 All|                Test|               Train|
+--------------------+--------------------+--------------------+--------------------+
|0087ff24898df109a...|[SOHFVJR12AF72A98...|[SOGBOQX12A8C139DF7]|[SOQRKXE12A8C138A...|
|0097e6d8f988dc58f...|[SOEDJNI12A58A80B...|                null|[SOEDJNI12A58A80B...|
+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows

In [25]:
union_all_required = union_all.withColumn('All_Test', news_udf(array('Test','All')))\
                    .withColumn('All-Test-Train', news_udf(array('Train', 'All_Test'))).select(['user_id','Test','Train','All-Test-Train'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
csv_buffer = StringIO()
union_all_required.toPandas().to_csv(csv_buffer)
client.put_object(Bucket=bucket, Key='union_all_required.csv',Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '406A68749C70557E', 'HostId': 'nt4JPBcSObvae1k/o2BDABxTLxy3Ju2eUKktuN4YtJ92Cgd8CQKMKfPR2EQcUZLuiDhMMA/oqZY=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'nt4JPBcSObvae1k/o2BDABxTLxy3Ju2eUKktuN4YtJ92Cgd8CQKMKfPR2EQcUZLuiDhMMA/oqZY=', 'x-amz-request-id': '406A68749C70557E', 'date': 'Sun, 12 Apr 2020 03:16:57 GMT', 'etag': '"235d0a38b75d3e119e405a25c160d558"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"235d0a38b75d3e119e405a25c160d558"'}

In [ ]:
csv_buffer = StringIO()
resultado.toPandas().to_csv(csv_buffer)
client.put_object(Bucket=bucket, Key='resultado_user_user.csv',Body=csv_buffer.getvalue())

# Song-Song

In [ ]:
jams_set_users = only_jams.groupby("song_id").agg(collect_set("user_id").alias('Set_users')).limit(5000)# Para cada cancion tenemos el grupo de usuarios que le gustan

jams_set_users_2 = jams_set_users.withColumnRenamed('song_id','song_id_2')

jams_set_users_2 = jams_set_users_2.withColumnRenamed('Set_users','Set_users_2')

def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

def news(s1,s2):
    return list(set(s2).difference(set(s1)))

jack = udf(lambda x: jaccard_similarity(x[0],x[1]), FloatType())

news_udf = udf(lambda y: news(y[0],y[1]), ArrayType(StringType()))

### El siguiente bloque es para solo calcular jaccard entre las canciones que esten el al menos el jamlist de 2 usuarios
only_jams2 = only_jams.withColumnRenamed('song_id','song_id_2')

double_jams = only_jams.join(only_jams2,on='user_id')

double_jams = double_jams.filter(double_jams.song_id!=double_jams.song_id_2)

initial_double_songs = double_jams.groupBy('song_id','song_id_2').agg(count('*').alias('number_common'))

initial_double_songs = initial_double_songs.filter(initial_double_songs.number_common>1)

initial_double_songs = initial_double_songs.select('song_id','song_id_2')

#####


double_users = initial_double_songs.join(jams_set_users, on = 'song_id').join(jams_set_users_2, on = 'song_id_2')## en el mismo dataframe ya tenemos la lista de usuarios a los que le gusta de cada par de cnaciones que nos interesan

double_users = double_users.filter(double_users.song_id!=double_users.song_id_2)

initial_rec=double_users.withColumn('Jackard', jack(array('Set_users', 'Set_users_2'))) ### Calculo jackard para las dos canciones

initial_rec = initial_rec.filter(initial_rec.Jackard>0) ### Solo me interesan los que le gustan

window = Window.partitionBy(initial_rec['song_id']).orderBy(initial_rec['Jackard'].desc())

same_song = initial_rec.select('*', rank().over(window).alias('rank')).\
                  filter(col('rank') <= 10).select('song_id','song_id_2','Jackard') ### Para cada cancion me quedo con las 10 más similares a ella

#same_song.toPandas.to_csv('song_song_similarity.csv') # Esto es lo más lento, por eso mejor guardarlo

user_jams = only_jams.groupby("user_id").agg(collect_set("song_id").alias('Set_songs')) ### Para cada usuario veo cual es su lista de jams

resultado = only_jams.join(same_song,on='song_id').groupby(["user_id","song_id_2"]).\
          agg(sum("Jackard").\
          alias('score_recomendacion')).\
          sort(desc("score_recomendacion")).\
          groupby("user_id").agg(collect_set("song_id_2").alias('Set_recomendaciones')).\
          join(user_jams, on = 'user_id').\
          withColumn('recomendacion', news_udf(array('Set_Songs', 'Set_recomendaciones'))).\
          select('user_id','recomendacion') ### segun las canciones que le gustan a cada persona, le recomiendo todo lo que "es igual" segun same songs y que no está en su jams list



In [ ]:
csv_buffer = StringIO()
resultado.toPandas().to_csv(csv_buffer)
client.put_object(Bucket=bucket, Key='resultado_item_item_2.csv',Body=csv_buffer.getvalue())